### 数据预处理

In [26]:
import pandas as pd
import re

In [27]:
dataset = pd.read_csv("data/bjsj.csv", engine='python', header=0, encoding='utf-8')

In [28]:
dataset.head()

,bjsj,bjnr,bjlbdm,bjlxdm,bjxldm,bjlbmc,bjlxmc,bjxlmc
0,2024-04-11 21:49:00.0,2024年4月11日 21时48分55秒 薛一铭( 180****5228 ，142427*...,10,100100,100120.0,刑事案件,盗窃,NaN
1,2024-04-11 21:43:25.0,2024年4月11日 21时43分22秒 郭女士( 139****6828 ) 报警：沙XX...,10,100100,100199.0,刑事案件,盗窃,NaN
2,2024-04-11 21:16:36.0,2024年4月11日 21时16分35秒 牛女士( 151****7579 ，142329*...,10,100100,100120.0,刑事案件,盗窃,NaN
3,2024-04-11 21:09:29.0,2024年4月11日 21时9分28秒 王先生( 151****8799、140111***...,10,100100,100120.0,刑事案件,盗窃,NaN
4,2024-04-11 21:01:51.0,2024年4月11日 10时44分48秒 黄志明( 151****3088 、350524*...,10,100100,100199.0,刑事案件,盗窃,NaN


In [29]:
if len(dataset.columns) > 7:
    dataset = dataset.drop(dataset.columns[7], axis=1)

# 判断是否已存在 'date' 和 'time' 列，避免重复执行
if 'date' not in dataset.columns and 'time' not in dataset.columns:
    # 拆分“bjsj”列成“日期”和“时间”两列，并转换为相应格式
    dataset['date'] = pd.to_datetime(dataset['bjsj'].str.split().str[0], format='%Y-%m-%d')
    dataset['time'] = pd.to_datetime(dataset['bjsj'].str.split().str[1], format='%H:%M:%S.%f').dt.time



def extract_info(text):
    # 使用正则表达式匹配关键词后面的内容作为报警内容
    match = re.search(r'(?:报警：|称|在)(.*)', text)
    if match:
        content = match.group(1)
        return content.strip()  # 去除首尾空格
    else:
        return text.strip()  # 如果没有匹配到关键词，则整行内容作为报警内容

# 判断是否已存在 'content' 列，避免重复执行
if 'content' not in dataset.columns:
    # 提取报警内容
    dataset['content'] = dataset['bjnr'].apply(extract_info)

dataset.head()

,bjsj,bjnr,bjlbdm,bjlxdm,bjxldm,bjlbmc,bjlxmc,date,time,content
0,2024-04-11 21:49:00.0,2024年4月11日 21时48分55秒 薛一铭( 180****5228 ，142427*...,10,100100,100120.0,刑事案件,盗窃,2024-04-11,21:49:00,XX村北XX街，女朋友放在口袋内一部价值9000元苹果手机被盗。
1,2024-04-11 21:43:25.0,2024年4月11日 21时43分22秒 郭女士( 139****6828 ) 报警：沙XX...,10,100100,100199.0,刑事案件,盗窃,2024-04-11,21:43:25,沙XX街天和XX小区6号楼3单元2802，现房门锁子打不开，门锁上有被砸痕迹，恐家内进入小偷...
2,2024-04-11 21:16:36.0,2024年4月11日 21时16分35秒 牛女士( 151****7579 ，142329*...,10,100100,100120.0,刑事案件,盗窃,2024-04-11,21:16:36,XX街寇XX路口往北200米路西处，放在口袋内一部价值1000余元的VIVO手机被盗。
3,2024-04-11 21:09:29.0,2024年4月11日 21时9分28秒 王先生( 151****8799、140111***...,10,100100,100120.0,刑事案件,盗窃,2024-04-11,21:09:29,亲XX街王府井百货门口，放在衣服口袋内价值17000余元的华为手机被盗。
4,2024-04-11 21:01:51.0,2024年4月11日 10时44分48秒 黄志明( 151****3088 、350524*...,10,100100,100199.0,刑事案件,盗窃,2024-04-11,21:01:51,昨天，将车停放在南中环当代城摩马门口，现发现三元催化被盗，价值35000元。


In [30]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 924 entries, 0 to 923
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   bjsj     924 non-null    object        
 1   bjnr     924 non-null    object        
 2   bjlbdm   924 non-null    int64         
 3   bjlxdm   924 non-null    int64         
 4   bjxldm   527 non-null    float64       
 5   bjlbmc   924 non-null    object        
 6   bjlxmc   924 non-null    object        
 7   date     924 non-null    datetime64[ns]
 8   time     924 non-null    object        
 9   content  924 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 72.3+ KB


In [31]:
# 去除重复列
dataset = dataset.drop_duplicates(subset=dataset.columns[1])

In [32]:
# 对文本进行增强
from nlpcda import Similarword, RandomDeleteChar

# 假设 dataset 是你的数据集 DataFrame
input_csv = dataset

# 初始化增强方法
smw = Similarword(create_num=2, change_rate=0.5)
rdc = RandomDeleteChar(create_num=2, change_rate=0.3)

# 定义函数来对文本进行增强
def augment_text(text):
    augmented_texts = []
    
    # 同义词替换增强
    for _ in range(2):  # 创建两个增强文本
        augmented_text_list = smw.replace(text)
        if len(augmented_text_list) > 1:
            augmented_text = augmented_text_list[1]  # 取第二个增强后的文本
            augmented_texts.append(augmented_text)
    
    # 随机字删除增强
    for _ in range(2):  # 创建两个增强文本
        augmented_text_list = rdc.replace(text)
        if len(augmented_text_list) > 1:
            augmented_text = augmented_text_list[1]  # 取第二个增强后的文本
            augmented_texts.append(augmented_text)
    
    return augmented_texts

# 对每行的 "content" 列进行增强，并保存增强后的数据
augmented_data = []

for index, row in input_csv.iterrows():
    original_content = row['content']
    augmented_contents = augment_text(original_content)
    
    for augmented_content in augmented_contents:
        new_row = row.copy()
        new_row['content'] = augmented_content
        augmented_data.append(new_row)

# 将增强后的数据转换为DataFrame，并保存为新的CSV文件
augmented_df = pd.DataFrame(augmented_data)


load :/Users/daypu/anaconda3/envs/pytorch/lib/python3.10/site-packages/nlpcda/data/同义词.txt done


In [33]:
augmented_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3122 entries, 0 to 923
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   bjsj     3122 non-null   object        
 1   bjnr     3122 non-null   object        
 2   bjlbdm   3122 non-null   int64         
 3   bjlxdm   3122 non-null   int64         
 4   bjxldm   1728 non-null   float64       
 5   bjlbmc   3122 non-null   object        
 6   bjlxmc   3122 non-null   object        
 7   date     3122 non-null   datetime64[ns]
 8   time     3122 non-null   object        
 9   content  3122 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 268.3+ KB


In [34]:
augmented_df["bjlbmc"].value_counts()

bjlbmc
治安警情      1078
刑事案件       484
交通警情       472
群众求助       352
社会联动       192
消防救援       164
举报线索       156
群体事件       154
其他报警类别      42
投诉监督        24
灾害事故         4
Name: count, dtype: int64

In [35]:
augmented_df["bjlxmc"].value_counts()

bjlxmc
诈骗             148
敲诈勒索           148
盗窃             144
打架斗殴            80
其它举报线索          80
环保执法            80
其它群众求助          80
其它交通管理          80
交通违法            80
交通事故            80
其它治安警情          80
交通设施            80
聚众上访            78
其它刑警情           76
提供线索            76
其它群体警情          76
色情淫秽            74
恐吓              72
失物求助            72
火灾              72
交通逃逸            72
其它消防救援          72
妨碍公务            70
扰乱秩序            70
治安纠纷            66
抢夺              64
其它社会联动          64
家庭暴力            64
强奸              64
交通秩序            60
抢劫              60
赌博              60
自杀求助            60
故意毁坏公私财物        58
走失求助            56
水、电、气、热险情求助     52
侵犯人身权利          48
其它报警类型          42
安全生产监督          32
非法侵入他人住宅        32
制贩、使用假币         20
开锁求助            20
抢险救援            20
交通保障            20
群众投诉            16
伤害              12
伪造票证、凭证         12
贩毒              12
绑架               8
坠楼求助             8
纵火               8
虐待               8
其他投诉监

In [36]:
# 保存修改后的CSV文件
augmented_df.to_csv('data/data_cleaned_enhanced.csv', index=False)

### 构建数据集

In [37]:
import torch
import numpy
import types
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import DataLoader, Subset

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

from datasets import Dataset

from transformers import BertTokenizer, BertConfig, BertModel, Trainer, TrainingArguments
from transformers import AdamW, BertForSequenceClassification, get_linear_schedule_with_warmup

from tqdm import tqdm, trange

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cpu')

In [39]:
# 读取数据

df = pd.read_csv("data/data_cleaned_enhanced.csv", header=0)

In [40]:
df.head()

,bjsj,bjnr,bjlbdm,bjlxdm,bjxldm,bjlbmc,bjlxmc,date,time,content
0,2024-04-11 21:49:00.0,2024年4月11日 21时48分55秒 薛一铭( 180****5228 ，142427*...,10,100100,100120.0,刑事案件,盗窃,2024-04-11,21:49:00,XX村北XX街，女朋友放在口袋内一部价9000元苹果手机被盗。
1,2024-04-11 21:49:00.0,2024年4月11日 21时48分55秒 薛一铭( 180****5228 ，142427*...,10,100100,100120.0,刑事案件,盗窃,2024-04-11,21:49:00,XX村北XX街，女朋友雄居口袋内一部价9000元苹手机被盗。
2,2024-04-11 21:49:00.0,2024年4月11日 21时48分55秒 薛一铭( 180****5228 ，142427*...,10,100100,100120.0,刑事案件,盗窃,2024-04-11,21:49:00,XX村北XX，女朋友放在口袋一部价值9000元苹果手机被盗
3,2024-04-11 21:49:00.0,2024年4月11日 21时48分55秒 薛一铭( 180****5228 ，142427*...,10,100100,100120.0,刑事案件,盗窃,2024-04-11,21:49:00,XX村北XX街女朋友放在口袋内一部价值9000元苹果手机被盗
4,2024-04-11 21:43:25.0,2024年4月11日 21时43分22秒 郭女士( 139****6828 ) 报警：沙XX...,10,100100,100199.0,刑事案件,盗窃,2024-04-11,21:43:25,沙XX街天和XX小区6号楼3单元2802，现房门锁子打不开，门锁上有被砸痕迹，恐家内进入鸡鸣...


In [41]:
major_vocab = df["bjlbmc"].unique()
minor_vocab = df["bjlxmc"].unique()

# 创建LabelEncoder对象
major_encoder = LabelEncoder()
minor_encoder = LabelEncoder()

# 对标签进行编码
df['major_label_encoded'] = major_encoder.fit_transform(df['bjlbmc'])
df['minor_label_encoded'] = minor_encoder.fit_transform(df['bjlxmc'])

# 计算主要标签和次要标签的类别数量
num_major_labels = df['bjlbmc'].nunique() # 大类
num_minor_labels = df['bjlxmc'].nunique() # 小类

# 设置 num_labels 为主要标签和次要标签类别数量之和
num_labels = num_major_labels + num_minor_labels
# 将各类标签保存到文本文件中
with open('labels_info.txt', 'w') as f:
    f.write(f"Number of major labels: {num_major_labels}\n")
    f.write(f"Number of minor labels: {num_minor_labels}\n")
    f.write(f"Total number of labels: {num_labels}\n")


import joblib
# 保存 LabelEncoder 对象
joblib.dump(major_encoder, 'major_encoder.pkl')
joblib.dump(minor_encoder, 'minor_encoder.pkl')

# 提取日期和时间特征
df['month'] = pd.to_datetime(df['date']).dt.month
df['hour'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.hour

In [42]:
# # 划分训练集和测试集
# train_texts, test_texts, train_major, test_major, train_minor, test_minor, train_month, test_month, train_hour, test_hour = train_test_split(
#     df['content'], df['major_label_encoded'], df['minor_label_encoded'], df['month'], df['hour'], test_size=0.2, random_state=42)

In [43]:
# 将数据按照每4行分组
n = 4
groups = [df.iloc[i:i+n] for i in range(0, len(df), n)]

# 将这些组转化为一个DataFrame列表
group_dfs = [group.reset_index(drop=True) for group in groups]

# 创建一个包含这些组的索引列表
group_indices = list(range(len(group_dfs)))

# 按照8:2的比例划分这些组的索引
train_indices, test_indices = train_test_split(group_indices, test_size=0.2, random_state=42)

# 根据划分的索引分别获取训练集和测试集
train_groups = [group_dfs[i] for i in train_indices]
test_groups = [group_dfs[i] for i in test_indices]

# 将这些组合并成训练集和测试集的DataFrame
train_df = pd.concat(train_groups).reset_index(drop=True)
test_df = pd.concat(test_groups).reset_index(drop=True)

# 提取训练集和测试集的各列数据
train_texts = train_df['content']
test_texts = test_df['content']
train_major = train_df['major_label_encoded']
test_major = test_df['major_label_encoded']
train_minor = train_df['minor_label_encoded']
test_minor = test_df['minor_label_encoded']
train_month = train_df['month']
test_month = test_df['month']
train_hour = train_df['hour']
test_hour = test_df['hour']

In [44]:
# 加载BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [45]:
# 创建训练集和测试集的Dataset对象
train_dataset = Dataset.from_pandas(pd.DataFrame({'text': train_texts, 'major_label': train_major, 'minor_label': train_minor, 'month': train_month, 'hour': train_hour}))
test_dataset = Dataset.from_pandas(pd.DataFrame({'text': test_texts, 'major_label': test_major, 'minor_label': test_minor, 'month': test_month, 'hour': test_hour}))

In [46]:
# 对文本进行tokenize，并转换为BERT的输入格式
def tokenize_texts(example):
    encoding = tokenizer(example['text'], padding='max_length', truncation=True, max_length=64)
    encoding['month'] = example['month']
    encoding['hour'] = example['hour']
    # encoding['minute'] = example['minute']
    return encoding

train_dataset = train_dataset.map(tokenize_texts, batched=True)
test_dataset = test_dataset.map(tokenize_texts, batched=True)

# 转换标签为张量类型
def convert_labels(example):
    example['major_label'] = torch.tensor(example['major_label'])
    example['minor_label'] = torch.tensor(example['minor_label'])
    example['month'] = torch.tensor(example['month'])
    example['hour'] = torch.tensor(example['hour'])
    # example['minute'] = torch.tensor(example['minute'])
    return example

train_dataset = train_dataset.map(convert_labels)
test_dataset = test_dataset.map(convert_labels)

# 设置返回张量格式
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'major_label', 'minor_label', 'month', 'hour'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'major_label', 'minor_label', 'month', 'hour'])

# 将数据集转换为 DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

Map:   0%|          | 0/2494 [00:00<?, ? examples/s]

Map:   0%|          | 0/628 [00:00<?, ? examples/s]

Map:   0%|          | 0/2494 [00:00<?, ? examples/s]

Map:   0%|          | 0/628 [00:00<?, ? examples/s]

### 训练模型

In [47]:
class CustomBertForSequenceClassification(BertForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size + 2, config.num_labels)  # 增加了2个特征

    def forward(self, input_ids=None, attention_mask=None, month=None, hour=None, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # BERT模型的pooler_output
        
        # 将时间特征拼接到pooled_output中
        time_features = torch.stack((month, hour), dim=1).float()  # 创建时间特征张量，假设 month 和 hour 的形状都是 [batch_size]
        pooled_output = torch.cat((pooled_output, time_features), dim=1)  # 在最后一个维度上拼接
        
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        
        outputs = (logits,) + outputs[2:]  # 将 logits 与 BERT 模型的其他输出组合在一起
        
        if labels is not None:
            if self.num_labels == 1:
                loss_fct = nn.MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs
        
        return outputs

In [48]:
# 加载BERT模型和训练参数
config = BertConfig.from_pretrained('bert-base-chinese', num_labels=num_labels)

model = CustomBertForSequenceClassification.from_pretrained('bert-base-chinese', config=config)  # 多标签分类，输出类别数量需适当调整
model.to(device)

Some weights of CustomBertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CustomBertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=

In [49]:
# # 定义优化器和损失函数
# optimizer = AdamW(model.parameters(), lr=4e-5, weight_decay=2e-3)
# criterion = nn.CrossEntropyLoss()  # 适用于多分类任务，根据实际情况可能需要调整损失函数

# # 训练循环
# num_epochs = 10
# for epoch in range(num_epochs):
#     # 训练模式
#     model.train()
#     train_loss = 0.0
#     for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         major_labels = batch['major_label'].to(device)
#         minor_labels = batch['minor_label'].to(device)
#         month = batch['month'].to(device)
#         hour = batch['hour'].to(device)

#         # 前向传播
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask, month=month, hour=hour)
#         logits = outputs[0]  # outputs包含损失和logits，第一个元素是logits

#         # 计算损失
#         loss_major = criterion(logits[:, :num_major_labels], major_labels)
#         loss_minor = criterion(logits[:, num_major_labels:], minor_labels)
#         loss = loss_major + loss_minor

#         # 反向传播和优化
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         train_loss += loss.item()

#     # 打印每个epoch的训练损失
#     print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss / len(train_loader)}")

#     # 评估模式
#     model.eval()
#     test_loss = 0.0
#     for batch in tqdm(test_loader, desc="Evaluating"):
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         major_labels = batch['major_label'].to(device)
#         minor_labels = batch['minor_label'].to(device)
#         month = batch['month'].to(device)
#         hour = batch['hour'].to(device)

#         # 前向传播
#         with torch.no_grad():
#             outputs = model(input_ids=input_ids, attention_mask=attention_mask, month=month, hour=hour)
#             logits = outputs[0]  # outputs包含损失和logits，第一个元素是logits

#             # 计算损失
#             loss_major = criterion(logits[:, :num_major_labels], major_labels)
#             loss_minor = criterion(logits[:, num_major_labels:], minor_labels)
#             loss = loss_major + loss_minor

#             test_loss += loss.item()

#     # 打印每个epoch的评估损失
#     print(f"Epoch {epoch + 1}/{num_epochs}, Test Loss: {test_loss / len(test_loader)}")

# # 保存模型
# torch.save(model.state_dict(), 'model.pth')

In [50]:
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

fold = 0
major_accs = []
minor_accs = []

for train_index, val_index in kf.split(group_indices):
    fold += 1
    train_groups = [group_dfs[i] for i in train_index]
    val_groups = [group_dfs[i] for i in val_index]

    train_df = pd.concat(train_groups).reset_index(drop=True)
    val_df = pd.concat(val_groups).reset_index(drop=True)

    # 提取训练集和验证集的各列数据
    train_texts = train_df['content']
    val_texts = val_df['content']
    train_major = train_df['major_label_encoded']
    val_major = val_df['major_label_encoded']
    train_minor = train_df['minor_label_encoded']
    val_minor = val_df['minor_label_encoded']
    train_month = train_df['month']
    val_month = val_df['month']
    train_hour = train_df['hour']
    val_hour = val_df['hour']

    # 创建训练集和验证集的Dataset对象
    train_dataset = Dataset.from_pandas(pd.DataFrame({'text': train_texts, 'major_label': train_major, 'minor_label': train_minor, 'month': train_month, 'hour': train_hour}))
    val_dataset = Dataset.from_pandas(pd.DataFrame({'text': val_texts, 'major_label': val_major, 'minor_label': val_minor, 'month': val_month, 'hour': val_hour}))

    # 对文本进行tokenize，并转换为BERT的输入格式
    train_dataset = train_dataset.map(tokenize_texts, batched=True)
    val_dataset = val_dataset.map(tokenize_texts, batched=True)

    # 转换标签为张量类型
    train_dataset = train_dataset.map(convert_labels)
    val_dataset = val_dataset.map(convert_labels)

    # 设置返回张量格式
    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'major_label', 'minor_label', 'month', 'hour'])
    val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'major_label', 'minor_label', 'month', 'hour'])

    # 将数据集转换为 DataLoader
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

    # 训练模型和评估
    model = CustomBertForSequenceClassification.from_pretrained('bert-base-chinese', config=config)
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=4e-5, weight_decay=2e-3)
    criterion = nn.CrossEntropyLoss()

    num_epochs = 3  # 演示目的，减少到3个epoch
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Fold {fold}, Epoch {epoch+1}/{num_epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            major_labels = batch['major_label'].to(device)
            minor_labels = batch['minor_label'].to(device)
            month = batch['month'].to(device)
            hour = batch['hour'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, month=month, hour=hour)
            logits = outputs[0]

            loss_major = criterion(logits[:, :num_major_labels], major_labels)
            loss_minor = criterion(logits[:, num_major_labels:], minor_labels)
            loss = loss_major + loss_minor

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        print(f"Fold {fold}, Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss / len(train_loader)}")

        model.eval()
        val_loss = 0.0
        val_preds = []
        val_labels = []
        for batch in tqdm(val_loader, desc=f"Fold {fold}, Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            major_labels = batch['major_label'].to(device)
            minor_labels = batch['minor_label'].to(device)
            month = batch['month'].to(device)
            hour = batch['hour'].to(device)

            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, month=month, hour=hour)
                logits = outputs[0]

                loss_major = criterion(logits[:, :num_major_labels], major_labels)
                loss_minor = criterion(logits[:, num_major_labels:], minor_labels)
                loss = loss_major + loss_minor

                val_loss += loss.item()

                # 计算准确率
                preds_major = logits[:, :num_major_labels].argmax(dim=1).detach().cpu().numpy()
                preds_minor = logits[:, num_major_labels:].argmax(dim=1).detach().cpu().numpy()
                val_preds.extend(list(zip(preds_major, preds_minor)))
                val_labels.extend(list(zip(major_labels.cpu().numpy(), minor_labels.cpu().numpy())))

        val_major_labels, val_minor_labels = zip(*val_labels)
        val_major_preds, val_minor_preds = zip(*val_preds)
        acc_major = accuracy_score(val_major_labels, val_major_preds)
        acc_minor = accuracy_score(val_minor_labels, val_minor_preds)
        print(f"Fold {fold}, Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss / len(val_loader)}, Major Accuracy: {acc_major}, Minor Accuracy: {acc_minor}")

        major_accs.append(acc_major)
        minor_accs.append(acc_minor)

# 输出所有折的平均准确率
print(f"Average Major Accuracy across all folds: {sum(major_accs) / len(major_accs)}")
print(f"Average Minor Accuracy across all folds: {sum(minor_accs) / len(minor_accs)}")

Map:   0%|          | 0/2806 [00:00<?, ? examples/s]

Map:   0%|          | 0/316 [00:00<?, ? examples/s]

Map:   0%|          | 0/2806 [00:00<?, ? examples/s]

Map:   0%|          | 0/316 [00:00<?, ? examples/s]

Some weights of CustomBertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/daypu/anaconda3/envs/pytorch/lib/python3.10/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Fold 1, Epoch 1/3:   7%|▋         | 23/351 [00:12<03:04,  1.78it/s]


KeyboardInterrupt: 